In [2]:
import pickle
import sys
import os
import json
import pandas as pd
import ast
import numpy as np
# add utils to path
sys.path.append('/root/capsule/code/beh_ephys_analysis')
from utils.beh_functions import parseSessionID

In [3]:
# Load the CCF summary CSV file
anti_file_path = '/root/capsule/code/data_management/antidromic_units.csv'
anti_df = pd.read_csv(anti_file_path)
anti_df['found'] = 0
# load the combined pickle file
combined_pickle_path = '/root/capsule/scratch/combined/combine_unit_tbl/combined_unit_tbl.pkl'
with open(combined_pickle_path, 'rb') as f:
    combined_unit_tbl = pickle.load(f)

In [8]:
skipping_ani_ids = []

In [9]:
for ind, row in anti_df.iterrows():
    session = row['session']
    unit = row['unit_id']
    anti = row['tier']
    proj = row['projection']
    aniID, date_obj, raw_id = parseSessionID(session)
    if aniID not in skipping_ani_ids:
        combined_unit_tbl.loc[(combined_unit_tbl['session'] == session) & (combined_unit_tbl['unit'] == unit), 'anti_resp'] = anti
        combined_unit_tbl.loc[(combined_unit_tbl['session'] == session) & (combined_unit_tbl['unit'] == unit), 'proj'] = proj
        anti_df.at[ind, 'found'] = 1

In [13]:
combined_unit_tbl.head()

,session,unit,qc_pass,opto_tagged,opto_tagging_pre,opto_tagging_post,anti,in_df,trial_count,p_max,...,wf,wf_aligned,wf_2d,probe,y_loc,rec_side,top,bottom,anti_resp,proj
0,ecephys_713854_2024-03-05_12-01-40,0.0,True,False,True,True,True,False,0,0.067565,...,"[-1.7105398178100586, -0.16496939957141876, 1....","[-1.7105398178100586, -0.16496939957141876, 1....","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",2,7.798922,-1,848.923865,722.03722,NaN,NaN
1,ecephys_713854_2024-03-05_12-01-40,1.0,True,False,True,True,True,False,0,0.133974,...,"[8.204042434692383, 8.311673164367676, 9.03707...","[8.204042434692383, 8.311673164367676, 9.03707...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",2,3.020706,-1,848.923865,722.03722,NaN,NaN
2,ecephys_713854_2024-03-05_12-01-40,2.0,True,False,True,True,True,False,0,-0.095125,...,"[-10.844730377197266, -10.085395812988281, -9....","[-10.844730377197266, -10.085395812988281, -9....","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",2,30.696420,-1,848.923865,722.03722,NaN,NaN
3,ecephys_713854_2024-03-05_12-01-40,3.0,True,False,True,True,True,False,0,0.030265,...,"[4.195621013641357, 5.438158988952637, 8.28008...","[4.195621013641357, 5.438158988952637, 8.28008...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",2,26.564554,-1,848.923865,722.03722,NaN,NaN
4,ecephys_713854_2024-03-05_12-01-40,4.0,True,False,True,True,True,False,0,0.113496,...,"[1.8883776664733887, 0.6048895716667175, 2.122...","[1.8883776664733887, 0.6048895716667175, 2.122...","[[1.9199707508087158, 1.4566508531570435, 1.16...",2,61.138142,-1,848.923865,722.03722,NaN,NaN


In [11]:
# update the existing pickle file with the new CCF coordinates
with open(combined_pickle_path, 'wb') as f:
    pickle.dump(combined_unit_tbl, f)

In [12]:
anti_df.to_csv(anti_file_path, index=False)